In [ ]:
!pip install torchmetrics
!pip install scipy
!pip install torch-fidelity

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
# root = r'/gdrive/My Drive/'

## set result_dir and gen_image_dir properly

In [ ]:
from torchmetrics.image.fid import FrechetInceptionDistance
from torchmetrics.image.inception import InceptionScore
from torchvision.io import read_image
import os, torch
#result_dir = r'/gdrive/MyDrive'
gen_image_dir = r'adaptive_sigmoid_20_200/sample_imgs'
gen_image_path = os.path.join(result_dir, gen_image_dir)
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print("genated image path: ", gen_image_path)
print("use device: ", device)

Load generated images

In [ ]:
image_names = os.listdir(gen_image_path)
image_list = []

for f in image_names:
    image_list.append(read_image(os.path.join(gen_image_path, f)).cuda())
gen_images = torch.stack(image_list)

Load real images

In [ ]:
from torchvision.datasets import CIFAR10
from torchvision.transforms import  Compose, ToTensor, Lambda


transform = Compose(
    [
        ToTensor(),
        Lambda(lambda x: x.to(device)),
    ]
)

cifar10_test = CIFAR10(root, train=False, transform = transform,download=True)

real_imgs = torch.from_numpy(cifar10_test.data).cuda()
real_imgs = real_imgs.permute(0,3,1,2)

Load FID evaluator

In [ ]:
fid = FrechetInceptionDistance(feature=2048).to(device)
inception_score = InceptionScore().to(device)
print("FID device: ", fid.device)
print("IS device: ", inception_score.device)

Update FID evaluator with data

In [ ]:
batch_size = 128
num_batches = (gen_images.shape)[0] // batch_size 
gen_images[:128].shape

for i in range(num_batches):
    fid.update(gen_images[batch_size*i:batch_size*(i+1)], real=False)
    inception_score.update(gen_images[batch_size*i:batch_size*(i+1)])

if gen_images.shape[0] % 128 != 0:
    fid.update(gen_images[batch_size*num_batches:], real=False)    
    inception_score.update(gen_images[batch_size*num_batches:])

num_batches = real_imgs.shape[0] // batch_size
for i in range(num_batches):
    fid.update(real_imgs[batch_size*i:batch_size*(i+1)], real=True)
fid.update(real_imgs[batch_size*num_batches:], real=True)

Compute FID score, IS




In [ ]:
fid_res = fid.compute()
is_res = inception_score.compute()
print(f"FID score: {fid_res.item()}")
print(f"Inception score: {is_res[0].item()}")



In [ ]:
from google.colab import runtime
runtime.unassign()